# Herramienta de Privacidad: IBM AI Privacy 360 Toolkit

## Introducción

En este notebook, utilizaremos el IBM AI Privacy 360 Toolkit para evaluar y mejorar la privacidad de un modelo de IA entrenado en el dataset de Heart Disease.

### Utilidad de IBM AI Privacy 360 Toolkit

El IBM AI Privacy 360 Toolkit es una herramienta diseñada para ayudar a los desarrolladores y científicos de datos a abordar las preocupaciones de privacidad en los modelos de IA. Proporciona métodos y técnicas para evaluar y mitigar el riesgo de violaciones de privacidad en los datos y los modelos.

El toolkit incluye funcionalidades para:

*   Evaluación de riesgos de privacidad.
*   Anonimización y agregación de datos.
*   Modelado y mitigación de riesgos.
*   Verificación de cumplimiento y auditoría.

En este notebook, utilizaremos el toolkit para evaluar y mejorar la privacidad de un modelo de IA entrenado en el dataset de Heart Disease.

### 1. Preparación del entorno

Antes de comenzar, asegurémonos de tener instaladas las bibliotecas necesarias e importadas las librerías.

Instalación de bibliotecas:

In [ ]:
!pip install ai-privacy-toolkit
!pip install adversarial-robustness-toolbox
!pip install -U scikit-learn

Importación de librerías:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from art.estimators.classification.scikitlearn import ScikitlearnDecisionTreeClassifier
from art.attacks.inference.attribute_inference import AttributeInferenceBlackBox

c:\Users\ALUMNO\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Carga del dataset

Comenzaremos cargando el dataset de Heart Disease.

In [2]:
data = pd.read_csv('../resources/HeartDiseaseDataset.csv')

## 3. Preprocesamiento de los datos

A continuación, dividiremos los datos en características (X) y etiquetas (y) y en conjuntos de entrenamiento y prueba. Tamibén realizaremos el preprocesamiento de los datos.

División de los datos:

In [3]:
# División de los datos
X = data.drop('target', axis=1)
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Preprocesamiento de los datos:

In [4]:
numeric_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']

preprocessor = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value=0))])

train_encoded = preprocessor.fit_transform(X_train)
test_encoded = preprocessor.transform(X_test)

## 4. Entrenamiento del modelo

Ahora estamos listos para entrenar nuestro modelo utilizando un árbol de decisión.

In [5]:
model = DecisionTreeClassifier()
model.fit(train_encoded, y_train)

art_classifier = ScikitlearnDecisionTreeClassifier(model)

base_model_accuracy = model.score(test_encoded, y_test)
print('Precisión del modelo base:', base_model_accuracy)

Precisión del modelo base: 0.819672131147541


## 5. Ataque de inferencia de atributos

Finalmente, utilizaremos un ataque de inferencia de atributos para demostrar la importancia de proteger la privacidad de los datos sensibles.

In [6]:
# Característica de ataque después del preprocesamiento
attack_feature = 1

# Datos de entrenamiento sin la característica atacada
x_train_for_attack = np.delete(train_encoded, attack_feature, 1)
# Solo la característica atacada
x_train_feature = train_encoded[:, attack_feature].copy().reshape(-1, 1)

bb_attack = AttributeInferenceBlackBox(art_classifier, attack_feature=attack_feature)

# Obtener predicciones del modelo original
x_train_predictions = np.array([np.argmax(arr) for arr in art_classifier.predict(train_encoded)]).reshape(-1, 1)

# Usar una porción del conjunto de entrenamiento para entrenar el ataque
attack_train_ratio = 0.8
attack_train_size = int(len(train_encoded) * attack_train_ratio)

# Entrenar el modelo de ataque
bb_attack.fit(train_encoded[:attack_train_size])

# Obtener valores inferidos
values = [0, 1]

inferred_train_bb = bb_attack.infer(x_train_for_attack[attack_train_size:], pred=x_train_predictions[attack_train_size:], values=values)

# Calcular precisión del ataque
attack_acc = np.sum(inferred_train_bb == np.around(x_train_feature[attack_train_size:], decimals=8).reshape(1, -1)) / len(inferred_train_bb)
print('Precisión del ataque de inferencia de atributos: {:.4f}'.format(attack_acc))

Precisión del ataque de inferencia de atributos: 0.7143


## Conclusiones

En este proyecto, hemos explorado el uso de IBM AI Privacy 360 para realizar un ataque de inferencia de atributos en un modelo de aprendizaje automático. A través de este proceso, hemos obtenido las siguientes conclusiones:

*   La privacidad es un aspecto crítico en el desarrollo de modelos de aprendizaje automático. Es fundamental proteger la información sensible de los individuos y considerar la privacidad desde el inicio.

*   Las técnicas de evaluación de privacidad, como el ataque de inferencia de atributos, nos permiten identificar las vulnerabilidades de privacidad en los modelos y evaluar su robustez frente a posibles ataques.

En resumen, este proyecto nos ha permitido explorar la importancia de la privacidad en el aprendizaje automático y utilizar herramientas como IBM AI Privacy 360 para evaluar y mitigar los riesgos asociados. Al integrar consideraciones de privacidad desde el inicio y adoptar un enfoque proactivo para proteger la información sensible, podemos desarrollar modelos de aprendizaje automático más éticos y responsables.